<a href="https://colab.research.google.com/github/barisetiawan51-code/LLM-RAG-Gemini_Contract-Hacktiv8/blob/main/LLM_RAG_Gemini_Hacktiv8_IBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Library

In [ ]:
!pip install faiss-cpu numpy
!pip install PyMuPDF
!pip install PyPDF2 tqdm pandas
!pip install faiss-cpu
!pip install pypdf
!pip install -U langchain langchain-google-genai google-generativeai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00


# Load Data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Understanding

In [ ]:
import os

folder_path = "/content/drive/MyDrive/Project Portofolio/LLM-RAG-Gemini-Hacktiv8/Dataset/Contracts_30_English/"
pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pdf")]

In [ ]:
import fitz

print(f"Total dokumen: {len(pdf_files)}")
print(pdf_files[:5])

Total dokumen: 30
['/content/drive/MyDrive/Project Portofolio/LLM-RAG-Gemini-Hacktiv8/Dataset/Contracts_30_English/Contract_14.pdf', '/content/drive/MyDrive/Project Portofolio/LLM-RAG-Gemini-Hacktiv8/Dataset/Contracts_30_English/Contract_17.pdf', '/content/drive/MyDrive/Project Portofolio/LLM-RAG-Gemini-Hacktiv8/Dataset/Contracts_30_English/Contract_4.pdf', '/content/drive/MyDrive/Project Portofolio/LLM-RAG-Gemini-Hacktiv8/Dataset/Contracts_30_English/Contract_29.pdf', '/content/drive/MyDrive/Project Portofolio/LLM-RAG-Gemini-Hacktiv8/Dataset/Contracts_30_English/Contract_27.pdf']


In [ ]:
# Baca sampel isi
with fitz.open(os.path.join(folder_path, pdf_files[0])) as doc:
    print(doc[0].get_text()[:500])

REAL ESTATE FINANCING AGREEMENT
BORROWER: Carlos Brown, residing at 456 Central Avenue - Los Angeles, CA, holder of SSN
nº 867542169.
LENDER: FINANCIAL BANK OF AMERICA Inc., registered under EIN nº 00-0000000,
headquartered at 1000 Main Avenue, New York, NY.
FINANCED AMOUNT: $1,203,432.00.
CLAUSE ONE – PURPOSE: This agreement aims to grant financing to the BORROWER for the
acquisition of the asset described in the preamble.
CLAUSE TWO – TERM: The financing period shall be up to 60 months, unless


# Data Preparation

In [ ]:
artifact_folder = "/content/drive/MyDrive/Project Portofolio/LLM-RAG-Gemini-Hacktiv8/LLM_RAG_artifacts/"
os.makedirs(artifact_folder, exist_ok=True)

In [ ]:
import PyPDF2, os, pandas as pd
from tqdm import tqdm

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

# Ekstrak semua PDF
data = []
for pdf in tqdm(pdf_files):
    with open(pdf, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page_num, page in enumerate(reader.pages, start=1):
            text = page.extract_text() or ""
            data.append({
                "filename": os.path.basename(pdf),
                "page_number": page_num,
                "text": text
            })

df = pd.DataFrame(data)
print(df.head())

# Split menjadi potongan kecil (chunk)
def chunk_text(text, chunk_size=1000, overlap=100):
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        start_idx = i
        end_idx = min(i + chunk_size, len(text))
        chunks.append((text[start_idx:end_idx], start_idx, end_idx))
    return chunks

chunks = []
for _, row in df.iterrows():
    for chunk_id, (chunk, start, end) in enumerate(chunk_text(row["text"])):
        chunks.append({
            "filename": row["filename"],
            "page_number": row["page_number"],
            "chunk_id": chunk_id,
            "start_idx": start,
            "end_idx": end,
            "text": chunk
        })

chunks_df = pd.DataFrame(chunks)

100%|██████████| 30/30 [00:09<00:00,  3.30it/s]

          filename  page_number  \
0  Contract_14.pdf            1   
1  Contract_14.pdf            2   
2  Contract_14.pdf            3   
3  Contract_17.pdf            1   
4  Contract_17.pdf            2   

                                                text  
0  REAL ESTATE FINANCING AGREEMENT\n \nBORROWER: ...  
1  CLAUSE ONE – PURPOSE: This agreement aims to g...  
2  CLAUSE ONE – PURPOSE: This agreement aims to g...  
3  STUDENT LOAN AGREEMENT\n \nBORROWER: Richard T...  
4  CLAUSE ONE – PURPOSE: This agreement aims to g...  


In [ ]:
chunks[5]

{'filename': 'Contract_14.pdf',
 'page_number': 3,
 'chunk_id': 1,
 'start_idx': 900,
 'end_idx': 1320,
 'text': 'will result in termination of the\nagreement and enforcement of the guarantees, regardless of notification.\nCLAUSE EIGHT – JURISDICTION: The parties elect the Courts of New York, NY, to settle any\ndisputes arising from this agreement.\nNew York, August 05, 2025\n_________________________________________\nCarlos Brown – BORROWER\n_________________________________________\nRepresentative – FINANCIAL BANK OF AMERICA (LENDER)\n'}

In [ ]:
print(f"✅ Jumlah potongan teks: {len(chunks_df)} dari {len(pdf_files)} file.")
chunks_df.head(10)

✅ Jumlah potongan teks: 180 dari 30 file.


,filename,page_number,chunk_id,start_idx,end_idx,text
0,Contract_14.pdf,1,0,0,1000,REAL ESTATE FINANCING AGREEMENT\n \nBORROWER: ...
1,Contract_14.pdf,1,1,900,1427,"e for the\nfinanced asset, with minimum covera..."
2,Contract_14.pdf,2,0,0,1000,CLAUSE ONE – PURPOSE: This agreement aims to g...
3,Contract_14.pdf,2,1,900,1134,will result in termination of the\nagreement a...
4,Contract_14.pdf,3,0,0,1000,CLAUSE ONE – PURPOSE: This agreement aims to g...
5,Contract_14.pdf,3,1,900,1320,will result in termination of the\nagreement a...
6,Contract_17.pdf,1,0,0,1000,STUDENT LOAN AGREEMENT\n \nBORROWER: Richard T...
7,Contract_17.pdf,1,1,900,1417,"financed asset, with minimum coverage as requi..."
8,Contract_17.pdf,2,0,0,1000,CLAUSE ONE – PURPOSE: This agreement aims to g...
9,Contract_17.pdf,2,1,900,1134,will result in termination of the\nagreement a...


# Modelling

In [ ]:
import os
import google.generativeai as genai
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("api_token_gemini_LLM")
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


In [ ]:
model_list = genai.list_models()
for m in model_list:
    print(m.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
mo

In [ ]:
embedding_model = "models/text-embedding-004"
llm_model = "models/gemini-2.0-flash"

In [ ]:
import google.generativeai as genai
import numpy as np
import faiss
import pickle

# Generate embedding untuk tiap chunk
embeddings = []
for text in tqdm(chunks_df["text"].tolist()):
    result = genai.embed_content(model=embedding_model, content=text)
    embeddings.append(result["embedding"])

embeddings = np.array(embeddings).astype("float32")

# Buat FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Simpan hasil (artifacts)
artifact_folder = "/content/drive/MyDrive/Project Portofolio/LLM-RAG-Gemini-Hacktiv8/LLM_RAG_artifacts/"
os.makedirs(artifact_folder, exist_ok=True)

faiss.write_index(index, os.path.join(artifact_folder, "faiss.index"))
np.save(os.path.join(artifact_folder, "embeddings.npy"), embeddings)
chunks_df.to_parquet(os.path.join(artifact_folder, "chunks.parquet"))
print("✅ Embedding & FAISS index saved.")


100%|██████████| 180/180 [02:14<00:00,  1.33it/s]


✅ Embedding & FAISS index saved.


In [ ]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

embedding_fn = GoogleGenerativeAIEmbeddings(model=embedding_model)
llm = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",     # atau "gemini-2.5-pro" jika ingin akurasi tinggi
    temperature=0.8,              # lebih tinggi = lebih ekspresif
    top_p=0.9,                    # menjaga variasi respons
    max_output_tokens=800,        # supaya jawabannya bisa agak panjang
    convert_system_message_to_human=True,  # biar gaya lebih natural
    verbose=True,                 # tampilkan log interaksi LangChain
)

In [ ]:
# Load Index dan Metadata
artifact_folder = "/content/drive/MyDrive/Project Portofolio/LLM-RAG-Gemini-Hacktiv8/LLM_RAG_artifacts/"
index = faiss.read_index(os.path.join(artifact_folder, "faiss.index"))
chunks_df = pd.read_parquet(os.path.join(artifact_folder, "chunks.parquet"))
embeddings = np.load(os.path.join(artifact_folder, "embeddings.npy"))

In [ ]:
import os
import numpy as np
import pandas as pd
import faiss
from tqdm import tqdm
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Retrieval (mengganti fungsi genai.embed_content → LangChain)
def retrieve_from_doc(query, file_name, top_k=5):
    doc_mask = chunks_df["filename"].str.lower() == file_name.lower()
    doc_chunks = chunks_df[doc_mask].reset_index(drop=True)
    doc_embeddings = embeddings[doc_mask]

    if len(doc_chunks) == 0:
        return []

    index_doc = faiss.IndexFlatL2(doc_embeddings.shape[1])
    index_doc.add(doc_embeddings.astype("float32"))

    query_emb = embedding_fn.embed_query(query)
    query_emb = np.array([query_emb]).astype("float32")

    distances, indices = index_doc.search(query_emb, top_k)

    results = []
    for idx in indices[0]:
        if 0 <= idx < len(doc_chunks):
            row = doc_chunks.iloc[idx]
            results.append({
                "text": row["text"],
                "page_number": row["page_number"],
                "chunk_id": row["chunk_id"],
                "start_idx": row["start_idx"],
                "end_idx": row["end_idx"]
            })
    return results


In [ ]:
def ask_gemini_rag(question, context):
    # ✨ Prompt kontekstual yang informatif dan human-like
    prompt = f"""
Anda adalah asisten hukum profesional namun komunikatif.
Analisis isi kontrak pembiayaan berikut untuk menjawab pertanyaan dengan jelas, padat, dan tidak terlalu kaku.

KONTEKS DOKUMEN:
{context}

PERTANYAAN:
{question}

Petunjuk:
- Gunakan informasi yang ada pada konteks secara langsung.
- Jangan membuat asumsi jika data tidak disebutkan.
- Jelaskan jawaban Anda dengan bahasa alami, profesional, dan mudah dipahami.
- Jika tidak ada informasi relevan, tulis dengan sopan bahwa data tersebut tidak tersedia dalam konteks.
"""

    # 🔷 Panggil LLM LangChain
    response = llm.invoke(prompt)

    # 🧠 Beberapa versi LangChain mengembalikan AIMessage, bukan .content
    if hasattr(response, "content"):
        return response.content.strip()
    elif hasattr(response, "text"):
        return response.text.strip()
    else:
        return str(response).strip()


## Evaluation

In [ ]:
import re

def clean_text(text):
    text = re.sub(r"\*\*(.*?)\*\*", r"\1", text)  # hapus bold markdown
    text = re.sub(r"\*", "", text)                 # hapus semua bintang tunggal
    text = re.sub(r"_+", "", text)                 # hapus garis bawah berlebih
    text = re.sub(r"\s{2,}", " ", text)            # ganti spasi ganda jadi tunggal
    text = re.sub(r"\n{2,}", "\n", text)           # rapikan baris kosong berlebih
    return text.strip()

# 🧠 Loop pertanyaan dengan hasil bersih
target_doc = "Contract_4.pdf"

questions = {
    "Pihak dan Identitas": [
        "Siapa nama pihak peminjam dan pemberi pinjaman dalam kontrak ini?",
        "Di mana alamat masing-masing pihak disebutkan?",
        "Apa jumlah total pembiayaan yang disetujui dalam perjanjian ini?",
    ],
    "Pembayaran dan Bunga": [
        "Bagaimana sistem pembayaran cicilan dijelaskan dalam perjanjian ini?",
        "Berapa tingkat bunga yang dikenakan setiap bulan?",
        "Apakah ada penyesuaian tahunan terhadap cicilan?",
    ],
    "Denda dan Kewajiban": [
        "Apa konsekuensi atau denda jika peminjam terlambat membayar cicilan?",
        "Bagaimana cara pemberi pinjaman dapat mengeksekusi jaminan jika terjadi gagal bayar?",
    ],
    "Asuransi dan Jaminan": [
        "Siapa yang wajib menanggung asuransi untuk aset yang dibiayai?",
        "Apa jenis jaminan yang diberikan oleh peminjam dalam kontrak ini?",
    ],
    "Hukum dan Yurisdiksi": [
        "Pengadilan mana yang berwenang menyelesaikan sengketa dalam perjanjian ini?",
        "Apakah kontrak ini dapat diakhiri sepihak oleh salah satu pihak?",
    ]
}

for section, qs in questions.items():
    print(f"\n📘 {section}")
    print("=" * 80)
    for q in qs:
        print(f"\n💬 Pertanyaan: {q}")
        docs = retrieve_from_doc(q, target_doc)

        if not docs:
            print("⚠️ Tidak ada konteks ditemukan untuk dokumen ini.\n")
            continue

        # Bersihkan teks sebelum dikirim ke Gemini
        context_text = "\n\n".join(clean_text(d) for d in docs)

        # Panggil fungsi RAG
        answer = ask_gemini_rag(q, context_text)

        # Bersihkan output model juga biar rapi
        answer_clean = clean_text(answer)
        print("🧠 Jawaban Gemini:\n", answer_clean)
        print("-" * 100)


📘 Pihak dan Identitas

💬 Pertanyaan: Siapa nama pihak peminjam dan pemberi pinjaman dalam kontrak ini?
🧠 Jawaban Gemini:
 Oke, berdasarkan informasi yang terdapat dalam kontrak pembiayaan yang Anda berikan, berikut adalah nama pihak peminjam dan pemberi pinjaman: Peminjam: Patricia Hall Pemberi Pinjaman: FINANCIAL BANK OF AMERICA Inc.
----------------------------------------------------------------------------------------------------

💬 Pertanyaan: Di mana alamat masing-masing pihak disebutkan?
🧠 Jawaban Gemini:
 Baik, mari kita lihat alamat masing-masing pihak dalam perjanjian pembiayaan ini: Peminjam (BORROWER): Patricia Hall, beralamat di 456 Central Avenue - Los Angeles, CA. Pemberi Pinjaman (LENDER): FINANCIAL BANK OF AMERICA Inc., berkantor pusat di 1000 Main Avenue, New York, NY.
----------------------------------------------------------------------------------------------------

💬 Pertanyaan: Apa jumlah total pembiayaan yang disetujui dalam perjanjian ini?
🧠 Jawaban Gemini:
 J

# Deployment with Streamlit

In [ ]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 87.1 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai

# Pilih model (misalnya Gemini 1.5 Flash — cepat dan gratis)
embedding_model = "models/text-embedding-004"
model = genai.GenerativeModel("models/gemini-2.0-flash")
model_list = genai.list_models()
for m in model_list:
    print(m.name)

models/embedding-gecko-001
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
mo

In [ ]:
%%writefile chatbot_contract_rag.py
import streamlit as st
import pandas as pd
import numpy as np
import os
import random
import faiss
import google.generativeai as genai

# ==========================
# 🔐 Konfigurasi API Key
# ==========================
genai.configure(api_key=os.getenv("api_token_gemini_LLM"))
embedding_model = "models/text-embedding-004"
model = genai.GenerativeModel("models/gemini-2.5-pro")

# ==========================
# ⚙️ Fungsi Retrieve & RAG
# ==========================
def retrieve(query, target_doc=None, top_k=3):
    query_emb = genai.embed_content(model=embedding_model, content=query)["embedding"]
    query_emb = np.array([query_emb]).astype("float32")
    distances, indices = index.search(query_emb, top_k)

    if target_doc:
        filtered = chunks_df[chunks_df["filename"] == target_doc]
        return filtered.iloc[indices[0]]["text"].tolist()
    else:
        return [chunks_df.iloc[i]["text"] for i in indices[0]]

def ask_gemini_rag(question, context, style="informative"):
    style_prompts = {
        "formal": "Gunakan gaya bahasa hukum yang formal dan profesional.",
        "natural": "Gunakan gaya bahasa alami seperti percakapan sehari-hari.",
        "ringkas": "Berikan jawaban singkat dan langsung ke poin utama.",
        "informative": "Berikan jawaban yang jelas, padat, dan disertai konteks tambahan bila relevan."
    }

    prompt = f"""
    Kamu adalah asisten hukum cerdas yang membantu menjelaskan isi kontrak pembiayaan.
    {style_prompts.get(style, style_prompts["informative"])}

    Berdasarkan konteks berikut:
    {context}

    Pertanyaan: {question}

    Jawaban dalam Bahasa Indonesia secara informatif.
    """

    response = model.generate_content(prompt)
    return response.text

# ==========================
# 🎯 Generator Pertanyaan
# ==========================
def generate_questions(topic, n=3):
    variations = {
        "pihak kontrak": [
            "Siapa pihak-pihak yang terlibat dalam perjanjian ini?",
            "Sebutkan siapa peminjam dan pemberi pinjaman dalam kontrak ini.",
            "Pihak mana saja yang disebutkan dalam kontrak pembiayaan ini?",
            "Siapa saja yang menandatangani perjanjian ini?",
            "Siapa pihak yang menerima pembiayaan dan pihak yang memberikan pembiayaan?"
        ],
        "pembayaran": [
            "Bagaimana sistem pembayaran diatur dalam kontrak ini?",
            "Berapa jangka waktu dan jumlah cicilan yang disepakati?",
            "Bagaimana ketentuan pembayaran dijelaskan oleh kontrak?",
            "Apakah pembayaran dilakukan setiap bulan atau sesuai kesepakatan tertentu?",
            "Ceritakan bagaimana proses pembayaran dijelaskan dalam perjanjian ini."
        ],
        "bunga": [
            "Berapa tingkat bunga yang ditetapkan dalam kontrak?",
            "Apakah terdapat bunga tambahan atau penyesuaian tahunan?",
            "Bagaimana suku bunga dihitung dalam perjanjian pembiayaan ini?",
            "Apakah tingkat bunga bersifat tetap atau berubah?",
            "Bagaimana cara penentuan bunga dijelaskan dalam kontrak?"
        ],
        "denda": [
            "Apa yang terjadi jika peminjam terlambat membayar cicilan?",
            "Apakah ada denda atau penalti atas keterlambatan pembayaran?",
            "Bagaimana kontrak mengatur konsekuensi keterlambatan pembayaran?",
            "Berapa besar denda yang dikenakan jika terjadi pelanggaran?",
            "Apakah kontrak menyebutkan sanksi terkait keterlambatan pembayaran?"
        ],
        "jaminan": [
            "Apa bentuk jaminan yang diberikan oleh peminjam?",
            "Bagaimana jaminan atau agunan diatur dalam kontrak ini?",
            "Siapa yang memegang hak atas jaminan sampai pembiayaan lunas?",
            "Apakah aset yang dibiayai dijadikan jaminan?",
            "Bagaimana proses eksekusi jaminan dijelaskan dalam perjanjian?"
        ],
        "hukum": [
            "Pengadilan mana yang berwenang menyelesaikan sengketa?",
            "Bagaimana kontrak mengatur yurisdiksi hukum?",
            "Apakah ada klausul yang menetapkan wilayah hukum tertentu?",
            "Jika terjadi sengketa, di mana perkara akan diselesaikan?",
            "Bagaimana ketentuan hukum dijelaskan dalam kontrak ini?"
        ]
    }
    if topic not in variations:
        return ["Topik tidak dikenal."]
    return random.sample(variations[topic], n)

# ==========================
# 🧠 Streamlit UI
# ==========================
st.title("📜 Legal Contract Chatbot (RAG)")

artifact_folder = "artifacts"
index = faiss.read_index(os.path.join(artifact_folder, "faiss.index"))
chunks_df = pd.read_parquet(os.path.join(artifact_folder, "chunks.parquet"))

contracts = sorted(chunks_df["filename"].unique())
target_doc = st.selectbox("Pilih kontrak yang ingin dianalisis:", contracts)

style = st.selectbox("Pilih gaya jawaban:", ["informative", "formal", "natural", "ringkas"])
mode = st.radio("Mode tanya:", ["Ketik pertanyaan sendiri", "Gunakan pertanyaan otomatis"])

if mode == "Ketik pertanyaan sendiri":
    user_question = st.text_input("Masukkan pertanyaan Anda:")
    if st.button("Tanyakan"):
        docs = retrieve(user_question, target_doc)
        context = "\n\n".join(docs)
        answer = ask_gemini_rag(user_question, context, style)
        st.markdown("### 💬 Jawaban:")
        st.write(answer)

else:
    topic = st.selectbox("Pilih topik:", ["pihak kontrak", "pembayaran", "bunga", "denda", "jaminan", "hukum"])
    if st.button("Generate pertanyaan"):
        auto_qs = generate_questions(topic)
        for q in auto_qs:
            st.markdown(f"**❓ {q}**")
            docs = retrieve(q, target_doc)
            context = "\n\n".join(docs)
            answer = ask_gemini_rag(q, context, style)
            st.markdown(f"🧠 **Jawaban:** {answer}")
            st.divider()


Writing chatbot_contract_rag.py


In [ ]:
!pip install pyngrok
from pyngrok import ngrok
!ngrok authtoken "32sKv0iM25HMW12YpC7JHXRGLrq_2mUnXGaYVXe82UfqD4FfM"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# Jalankan Streamlit di background
!streamlit run chatbot_contract_rag.py --server.port 8501 &

# Buat tunnel agar bisa diakses dari luar
public_url = ngrok.connect(8501)
public_url





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.57.139.250:8501

  Stopping...


<NgrokTunnel: "https://75d323a543ae.ngrok-free.app" -> "http://localhost:8501">